#Installs

In [1]:
!pip install -U gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 8.9 MB/s eta 0:0

In [12]:
import gradio as gr
import torch
import urllib.request
import sqlite3
import pandas as pd

# Model loading and setup

In [10]:


# Clone the repository to get the saved model
urllib.request.urlretrieve('https://github.com/CNielsen94/NN_exercises_AAUBSDS/raw/main/MLOps_assignments/Assignment_one/model.pth', 'model.pth')

# Load the saved model state dictionary
model_state_dict = torch.load('model.pth', map_location=torch.device('cpu'))

# Define the model architecture
model_net1 = torch.nn.Sequential(torch.nn.Linear(3,2),
                                 torch.nn.ReLU(),
                                 torch.nn.Linear(2,1),
                                 torch.nn.Sigmoid())

# Load the saved model state dictionary into the model architecture
model_net1.load_state_dict(model_state_dict)

<All keys matched successfully>

# First gradio interface for prediction (No DB implemented, and possibly faulty)

In [133]:
def first_predict(TotalWorkingYear_tier, EnvironmentSatisfaction, YearsAtCompany_tier):
    inputs = torch.tensor([int(TotalWorkingYear_tier), int(EnvironmentSatisfaction), int(YearsAtCompany_tier)])
    output = model_net1(inputs.float())
    prediction = output.item()
    if prediction < 0.5:
        return "No Attrition"
    else:
        return "Attrition"

In [128]:
predict(1,2,5)

'No'

In [129]:
inputs = [
    gr.inputs.Slider(minimum=0, maximum=5, step=1, default=2, label="TotalWorkingYear_tier"),
    gr.inputs.Slider(minimum=1, maximum=4, step=1, default=2, label="EnvironmentSatisfaction"),
    gr.inputs.Slider(minimum=0, maximum=5, step=1, default=2, label="YearsAtCompany_tier")
]

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [130]:
outputs = gr.outputs.Label(num_top_classes=1)

/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [134]:
gr.Interface(fn=first_predict, inputs=inputs, outputs=outputs, title="Attrition Predictor").launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

So our predictions seem to always be zero, which is most likely due to overfitting and or just a too simple network. Let's integrate some database fetching for individual employees anyways:

# Second interface with DB interaction

Fetching HR_DB.db from repository:

In [32]:
url = 'https://github.com/CNielsen94/NN_exercises_AAUBSDS/raw/main/MLOps_assignments/Assignment_one/HR_DB.db'
filename = 'HR_DB.db'

urllib.request.urlretrieve(url, filename)

('HR_DB.db', <http.client.HTTPMessage at 0x7fa7c423c670>)

In [135]:
def predict_attrition(EmployeeID):

    # Create a new connection and cursor object
    conn = sqlite3.connect('HR_DB.db')
    cursor = conn.cursor()

    # Get the Employee's TotalWorkingYears_tier and YearsAtCompany_tier from EmployeeTiers table
    cursor.execute(f"SELECT TotalWorkingYear_tier, YearsAtCompany_tier FROM EmployeeTiers WHERE EmployeeID={EmployeeID}")
    tiers = cursor.fetchone()

    # Get the Employee's EnvironmentSatisfaction from EmployeeSurvey table
    cursor.execute(f"SELECT EnvironmentSatisfaction FROM employee_survey WHERE EmployeeID={EmployeeID}")
    env_satisfaction = cursor.fetchone()

    # Convert the fetched data into a tensor
    inputs = torch.tensor([tiers[0], env_satisfaction[0], tiers[1]], dtype=torch.float)

    # Make the prediction using the loaded model
    output = model_net1(inputs).item()
    prediction = "Attrition" if output > 0.5 else "No Attrition"
    message = f"The predicted Attrition for EmployeeID {EmployeeID} is: {prediction}"

    # Close the cursor and connection objects
    cursor.close()
    conn.close()

    return prediction

In [46]:
#query = 'SELECT * FROM EmployeeTiers'
#df = pd.read_sql(query, conn)

Quick test for predictions. Our model is seemingly bad at predicting the results, as all employees seem to love the workplace.

In [137]:
print(predict_attrition(382))

No Attrition


In [136]:
# Create the Gradio interface
iface = gr.Interface(
    fn=predict_attrition, 
    inputs=gr.inputs.Number(label="Employee ID"), 
    outputs=gr.outputs.Textbox(label="Attrition Prediction"),
    title="Employee Attrition Prediction",
    description="Enter an EmployeeID to predict their likelihood of attrition."
)

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


In [138]:
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>